Website: https://sg.carousell.com/ 

Item: Baby Chair 

Condition: New 

Seller’s location: Within 5km from Woodlands 

Price Range: <span>&#36;</span>5 to <span>&#36;</span>150 

Seller’s Ratings: The more stars the better. The maximum number of stars is five. 

The results should be returned from the most recent first as we prefer to buy latest posted item. 
The results should include <b>Date</b>, <b>title</b>, <b>link</b>, <b>price</b>, <b>seller</b>, <b>seller's link</b> and <b>seller’s ratings</b>.  


Example:
https://sg.carousell.com/search/baby%20chair?condition_v2=NEW&location_name=Woodlands&price_end=150&price_start=0&range=5&sort_by=time_created%2Cdescending

<p>    
    
References: <br />
https://selenium-python.readthedocs.io/index.html
    
<p>
Notes: <br />
print(self.driver.page_source) # Get page source via Selenium

In [65]:
# TODO:
# 1. Send Key to Search Box Input Type=Search
# OK 2. Save page source
# 3. Generate csv of <b>Date</b>, <b>title</b>, <b>link</b>, <b>price</b>, <b>seller</b>, <b>seller's link</b> and <b>seller’s ratings</b>
# OK 4. Clean up links returned
# OK 5. Format Date time including Spotlight
# 6. Retrieve Seller's Rating
# OK 7. Filter items with title without "Baby Chair"
# OK 8. Screenshot of main page
# 9. Load More

In [66]:
from datetime import datetime, timedelta

t = datetime.now() - timedelta(hours=3)

In [67]:
t

datetime.datetime(2020, 3, 3, 13, 22, 27, 618880)

In [68]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib.request import Request

from datetime import datetime,timedelta
import csv
import os
import re

In [69]:
class CarousellScraper(object):
    def __init__(self,item='baby chair', condition='NEW', location='Woodlands', 
                 distance='5', min_price='0', max_price='150'):
        
        # For data logging
        self.curdatetime = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.item      = item
        self.condition = condition
        self.location  = location
        self.distance  = distance
        self.min_price = min_price
        self.max_price = max_price
        self.base_url  = 'https://sg.carousell.com'

        self.url = f"{self.base_url}/search/{item}?condition_v2={condition}&location_name={location}&price_end=\
{max_price}&price_start={min_price}&range={distance}&sort_by=time_created,descending"
        print(self.url)
        
        self.driver = webdriver.Chrome(executable_path='chromedriver.exe')
        self.delay = 3
        
        #Create folder for data logging
        if not os.path.exists("raw"):
            os.mkdir("raw")
        
        if not os.path.exists("processed"):
            os.mkdir("processed")
        
    def load_carousell_url(self):
        
        self.driver.get(self.url)
        
        try:
            wait = WebDriverWait(self.driver, self.delay)
            # Search box
            wait.until(EC.presence_of_element_located((By.XPATH,"//input[@type='search']" )))
            
            #Save page source
            dest_path = "raw/"+self.curdatetime+"_CarousellSource.html"
            with open(dest_path, "w+", encoding="utf-8") as f:
                f.write(self.driver.page_source)

        except TimeOutException:
            print("Time out to load ", self.url)
        
        ### image saved in raw folder
        screenshot_path = "raw/"+self.curdatetime+"_CarousellSearch.png"
        self.driver.get_screenshot_as_file(screenshot_path) 
     
    def extract_item_information(self):
        pass
    
#    The results should include <b>Date</>, <b>title</b>, <b>link</b>, <b>price</b>, <b>seller<b> and <b>seller’s ratings<b/>.  

    def extract_item_title(self):
        
        print()
        headers = {}
        headers['User-Agent'] = "Chrome/24.0.1312.27"
        request = Request(self.url, headers = headers)
        html = urlopen(request).read()
        soup = bs(html, 'lxml')
        
        #The items are in three layers below main tag
        sellers      = []
        seller_links = []
        dates        = []
        titles       = []
        item_links   = []
        prices       = []
        
        items = soup.find("main").find_all('div')[0].find_all('div')[0].children
        for item in items:
            base_items = item.find_all('a')
            if (len(base_items) > 0):
                # seller, seller's page, date time
                base_items01 = base_items[0]
#                 print(base_items01)
                seller      = base_items01.find_all('p')[0].text
        
                seller_link = self.shorten_url(self.base_url + base_items01['href'])
                
                date        = base_items01.find_all('p')[-1].text
                print("Date:" + date)
                date = self.return_date(date)
                print("Date:" + date)
                print("Seller:" +  seller)
                print("Seller_Link:" +  seller_link)
                
                base_items02 = base_items[1]
                item_link = self.shorten_url(self.base_url + base_items02['href'])
                title = base_items02.find_all('p')[0].text
                price = base_items02.find_all('p')[1].text
                
                # Only match item's name containing 'baby' and 'chair' (case insensitive)
                if re.search('(?i)(baby.*chair)', title) is not None:
                    sellers.append(seller)
                    seller_links.append(seller_link)
                    dates.append(date)
                    item_links.append(item_link)
                    titles.append(title)
                    prices.append(price)
                print("\nTitle:" + title  + "\nLink:" + item_link + "\nPrice:" + price)
                print()
                print()
        
        print("sellers:", len(sellers), ", seller_links:", len(seller_links), ", dates: ", len(dates), 
", titles: ", len(titles), ", title links: ", len(item_links), ", prices: ", len(prices))
        
        # Write to csv
        dest_path = "processed/"+self.curdatetime+"_Carousell_Search_"+self.item+".csv"
        dest_path = dest_path.replace(' ', '')
        csvFile = open(dest_path, 'w+', encoding='utf-8',newline='')
        try:
            writer = csv.writer(csvFile)
            writer.writerow(('Date', 'Item', 'Item_Link', 'Price', 'Seller', 'Seller_Link'))
            for i in range(len(dates)):
                writer.writerow((dates[i].strip(), titles[i].strip(), item_links[i].strip(), prices[i].strip(), 
sellers[i].strip(), seller_links[i].strip()))
        finally:
            csvFile.close()
    
    def return_date(self, d):
        # https://docs.python.org/3/library/datetime.html?highlight=datetime#datetime.timedelta
        # date2 = date1 + timedelta
        # class datetime.timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)¶
        #re.sub(r"(\w)(\w+)(\w)", repl, text)
        if 'hour' in d:
            hour = re.sub(r"[a-zA-Z]", "", d)
            t = datetime.now() - timedelta(hours=int(hour))
            posted_date = t.strftime('%Y-%m-%d')
            print('posted_date:' + posted_date)
        elif 'day' in d:
            day = re.sub(r"[a-zA-Z]", "", d)
            t = datetime.now() - timedelta(days=int(day))
            posted_date = t.strftime('%Y-%m-%d')
            print('posted_date:' + posted_date)
        elif 'minute' in d:
            minute = re.sub(r"[a-zA-Z]", "", d)
            t = datetime.now() - timedelta(days=int(minute))
            posted_date = t.strftime('%Y-%m-%d')
            print('posted_date:' + posted_date)
        else:
            posted_date = d
     
        return posted_date
   
        
    
    def shorten_url(self, url):
        pos = url.find('?')
        if pos != -1:
            return url[:pos]
        else:
            return url
 
    
    def extract_item_url(self):
        pass
    
    def extract_item_price(self):
        pass
    
    def extract_item_seller(self):
        pass
    
    def extract_item_seller_ratings(self):
        pass
    
    def quit(self):
        self.driver.close()
 
 
 
 

In [70]:
scraper = CarousellScraper()

https://sg.carousell.com/search/baby chair?condition_v2=NEW&location_name=Woodlands&price_end=150&price_start=0&range=5&sort_by=time_created,descending


In [71]:
scraper.load_carousell_url()

In [72]:
scraper.extract_item_title()


Date:Spotlight
Date:Spotlight
Seller:haus_vlog
Seller_Link:https://sg.carousell.com/haus_vlog/

Title:Velvet Lounge Chair
Link:https://sg.carousell.com/p/velvet-lounge-chair-280178428
Price:S$509


Date:2 hours ago
posted_date:2020-03-03
Date:2020-03-03
Seller:desmondlim01238
Seller_Link:https://sg.carousell.com/desmondlim01238/

Title:Ceilling hook instsll
Link:https://sg.carousell.com/p/ceilling-hook-instsll-230784946
Price:S$50


Date:4 hours ago
posted_date:2020-03-03
Date:2020-03-03
Seller:momlove
Seller_Link:https://sg.carousell.com/momlove/

Title:🔥1 Day Sale Only! ❤️[🇺🇸Imported] Ready Stock❤️ Brand New Authentic Ingenuity Baby Base 2-in-1 Baby Booster Seat Feeding Chair with Tray (Various Colour Available) Premium Quality
Link:https://sg.carousell.com/p/🔥1-day-sale-only-❤️-🇺🇸imported-ready-stock❤️-brand-new-authentic-ingenuity-baby-base-2-in-1-baby-booster-seat-feeding-chair-with-tray-various-colour-available-premium-quality-278686700
Price:S$62


Date:5 hours ago
posted_date:

In [ ]:
#scraper.quit()